In [1]:
# import libraries
import time
import os
import re
from pymongo import MongoClient
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk  
import numpy as np  
import random  
import string

import bs4 as bs  
import urllib.request  
import re 

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import heapq

In [2]:
# check working directory
os.getcwd()

'C:\\Users\\aksha\\Programming_And_Course\\UCD\\Machine_Learning\\Final Project'

### Getting data from Mongodb

In [3]:
# mongo db params
client = MongoClient("mongodb://localhost:27017/")
database = client["ddr"]
collection = database["goodreads_books_reviews"]

In [4]:
# get data from mongodb
def fetch_data(collection_name, df):
    for item in collection_name.find():
        try:
            bookTitle = item['bookTitle']
            review_text = ''
            
            for review in item['reviewsInfo']:
                review_text += "".join(review['reviewerText'])
                
            df = df.append([[bookTitle, review_text]],
                           ignore_index = True)
        
        except:
            pass
        
    return(df)

In [5]:
# data frame
df_book_info = pd.DataFrame()
df_book = fetch_data(collection, df_book_info)

# renaming columns
df_book.columns = ['bookTitle','reviewText']

# dropping duplicates
df_book = df_book.drop_duplicates(subset='bookTitle',ignore_index=True)
df_book = df_book[df_book['bookTitle'] != '']
df_book


,bookTitle,reviewText
0,The New Drawing on the Right Side of the Brain,This book is a double-edged sword: On one side...
1,Steal Like an Artist: 10 Things Nobody Told Yo...,These kind of books are like candy. They make ...
2,Wall and Piece,"Everyone in Britain knows Banksy, but I'm ofte..."
3,The Artist's Way: A Spiritual Path to Higher C...,Julia Cameron works my last nerve. She's alway...
5,The Letters of Vincent van Gogh,STARRY LETTERS\nIn my youth I felt saturated w...
...,...,...
873,Kon-Tiki,Every Norwegian family we knew had a copy of t...
874,Tracks: A Woman's Solo Trek Across 1700 Miles ...,I was disappointed by this book. I felt that t...
875,The Old Ways: A Journey on Foot,This one really hit the sweet spot for me. It ...
876,Holy Cow: An Indian Adventure,I have to admit that I decided to read this bo...


### Transforming reviews using bag of words method

In [6]:
###################### Transforming Reviews ################

# Calling DataFrame constructor on list
df = pd.DataFrame(df_book,columns =['reviewText'])

#library that contains punctuation
import string
string.punctuation

#defining the function to remove punctuation
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree
#storing the puntuation free text
df['Description_Token'] = df['reviewText'].apply(lambda x:remove_punctuation(x))
df['Description_Token']= df['Description_Token'].apply(lambda x: x.lower())

# using list comprehension + split()
# Tokenizing strings in list of strings
df['Description_Token'] = [sub.split() for sub in df['Description_Token']]

#importing nlp library
import nltk
#Stop words present in the library
stopwords = nltk.corpus.stopwords.words('english')

#defining the function to remove stopwords from tokenized text
def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output

#applying the function
df['Description_Token']= df['Description_Token'].apply(lambda x:remove_stopwords(x))

#importing the Stemming function from nltk library
from nltk.stem.porter import PorterStemmer
#defining the object for stemming
porter_stemmer = PorterStemmer()

#defining a function for stemming
def stemming(text):
    stem_text = [porter_stemmer.stem(word) for word in text]
    return stem_text


df['Description_Token']=df['Description_Token'].apply(lambda x: stemming(x))

from nltk.stem import WordNetLemmatizer
#defining the object for Lemmatization
wordnet_lemmatizer = WordNetLemmatizer()

import nltk
#nltk.download()
nltk.download('wordnet')
nltk.download('omw-1.4')

#defining the function for lemmatization
def lemmatizer(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text

df['Description_Token']=df['Description_Token'].apply(lambda x:lemmatizer(x))

import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize.treebank import TreebankWordDetokenizer

df['Description_Token'] = [' '.join(x) for x in df['Description_Token']] 



[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aksha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\aksha\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [7]:
count = CountVectorizer(lowercase=True, max_features=20000,ngram_range=(2,4))
count_matrix = count.fit_transform(df['Description_Token'])
df_bow_sklearn = pd.DataFrame(count_matrix.toarray(),columns=count.get_feature_names())
df_bow_sklearn['bow_token'] = df_bow_sklearn.values.tolist()

df_book['bow_token'] = df_bow_sklearn['bow_token']

c:\users\aksha\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [8]:
df_book = df_book.dropna(axis=0)
df_book

,bookTitle,reviewText,bow_token
0,The New Drawing on the Right Side of the Brain,This book is a double-edged sword: On one side...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,Steal Like an Artist: 10 Things Nobody Told Yo...,These kind of books are like candy. They make ...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Wall and Piece,"Everyone in Britain knows Banksy, but I'm ofte...","[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,The Artist's Way: A Spiritual Path to Higher C...,Julia Cameron works my last nerve. She's alway...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,The Letters of Vincent van Gogh,STARRY LETTERS\nIn my youth I felt saturated w...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...
872,Bill Bryson's African Diary,All the Bryson goodness you've come to know an...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
873,Kon-Tiki,Every Norwegian family we knew had a copy of t...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
874,Tracks: A Woman's Solo Trek Across 1700 Miles ...,I was disappointed by this book. I felt that t...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
875,The Old Ways: A Journey on Foot,This one really hit the sweet spot for me. It ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [9]:
len(df_book['bow_token'][0])

20000

### Similarity Function

In [10]:
# similarity function
from scipy import spatial

def Similarity(a,book2):
    a = a.iloc[0,:]
    b = df_book.iloc[book2]
       
    descA = a['bow_token']
    descB = b['bow_token']
    descDistance = spatial.distance.cosine(descA, descB) 
    
    return descDistance

import operator

def predict_score():
    name = input('Enter a book title: ')
    
    new_book = df_book[df_book['bookTitle'].str.contains(name)]
    print('Selected Movie: ',new_book.bookTitle.values[0])
    
    def getNeighbors(baseBook, K):
        distances = []
        
        for index in range(len(df_book)):
            if df_book.iloc[index,:]['bookTitle'] != baseBook['bookTitle'].values[0]:
                dist = Similarity(baseBook, index)
                distances.append((df_book.iloc[index,:]['bookTitle'], dist))
                    
        distances.sort(key=operator.itemgetter(1))
        # initialize neighbors
        neighbors = []
        
        # get first 10 neighbors
        for x in range(K):
            neighbors.append(distances[x])
        return neighbors
    
    
    # initilaize k
    K = 10
    neighbors = getNeighbors(new_book, K)
    
    print('\nRecommended Movies: \n')
    
    for neighbor in neighbors:
        print(neighbor[0])
    
    print('\n')

### Test Output

In [11]:
predict_score()

Enter a book title: Thinking
Selected Movie:  Thinking, Fast and Slow

Recommended Movies: 

Waking Up: A Guide to Spirituality Without Religion
Influence: The Psychology of Persuasion
City of Bones
Who Moved My Cheese?
Measure What Matters
The Well of Ascension
Eleanor & Park
A Clash of Kings
The Elite
Rework


